In [1]:
# Importando bibliotecas
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
# Classe responsável por armazenar os dados de folhosas e inmet por csv
class AllDados:
    def __init__(self, folhosas_csv, inmet_sp_csv, inmet_sorocaba_csv):
        # Armazenando os dados da tabela csv em um DataFrame pandas
        self.folhosas = pd.read_csv(folhosas_csv)
        self.inmet_sp = pd.read_csv(inmet_sp_csv)
        self.inmet_sorocaba = pd.read_csv(inmet_sorocaba_csv)

In [3]:
# Classe responsável por processar dados
class ProcessadorDados:    
    # Método responsável por limpar outliers
    def clean_outliers(df):
        df = df[(df != -9999).all(axis=1)]
        df.loc[df.eq(-9999).any(axis=1)]
        
        return df

    # Método responsável por agrupar dados por Data para descobrir a média diária de cada coluna
    def daily_groupby(df):
        df = df.groupby(['Data']).mean()
        df.reset_index(inplace=True)

        return df
    
    # Método responsável por extrair as datas de catálogo de produtos
    def date_extract(df):
        # Selecionando datas únicas
        datas = df['Data'].unique()

        # Encontrando a data anterior a uma semana do primeiro produto
        primeira_data = pd.Timestamp(datas[0]) - dt.timedelta(days=7)

        # Adicionando a primeira data as datas descobertas anteriormente
        datas = pd.Series(datas)
        datas = pd.concat([pd.Series([primeira_data]), datas], ignore_index=True)
        
        return datas

    # Método responsável por agrupar dados pelos intervalos das datas de catálogo descobrir as médias de cada coluna
    def weekly_groupby(inmet, datas):
        # Criando um DataFrame para armazenar os dados do inmet
        inmet_weekly = pd.DataFrame()

        # Foreach para percorrer as datas identificadas e realizar os processamentos de dados
        for i in range(len(datas) - 1):
            intervalo_datas = inmet[(inmet['Data'] >= datas[i]) & (inmet['Data'] <= datas[i + 1])]

            pivot = intervalo_datas[
                ['Média Precipitação Total (mm)', 'Média Pressão Atmosférica (mB)', 'Média Temperatura do Ar (°C)',
                 'Média Umidade Relativa do Ar (%)', 'Média Vento Velocidade (m/s)']].mean()
            
            pivot['Data'] = datas[i + 1]
            
            inmet_weekly = inmet_weekly.append(pivot, ignore_index=True)

        return inmet_weekly
    
    # Método responsável por executar os processamentos de dados e gerar um novo inmet
    def process_inmet(folhosas, inmet):
        inmet = ProcessadorDados.clean_outliers(inmet)
        inmet['Data'] = pd.to_datetime(inmet['Data'])

        datas = ProcessadorDados.date_extract(folhosas)
        inmet_processed = ProcessadorDados.weekly_groupby(inmet, datas)

        return inmet_processed
    
    # Método responsável por formatar colunas DataFrame(arredondamento de casas decimais)
    def format_columns(df):
        df['Média Precipitação Total (mm)'] = df['Média Precipitação Total (mm)'].round(4)
        df['Média Pressão Atmosférica (mB)'] = df['Média Pressão Atmosférica (mB)'].round(3)
        df['Média Temperatura do Ar (°C)'] = df['Média Temperatura do Ar (°C)'].round(2)
        df['Média Umidade Relativa do Ar (%)'] = df['Média Umidade Relativa do Ar (%)'].round(2)
        df['Média Vento Velocidade (m/s)'] = df['Média Vento Velocidade (m/s)'].round(2)
        df['Média Umidade Relativa do Ar (%)'] = df['Média Umidade Relativa do Ar (%)'].astype(str) + '%'

        return df
    
    # Método responsável por fundir os dados de folhosas e inmet
    def merge_folhosas_inmet(folhosas, inmet):
        # Merge
        folhosas_inmet = pd.merge(folhosas, inmet)

        # Retorna DataFrame formatado
        return ProcessadorDados.format_columns(folhosas_inmet)

In [4]:
# Classe responsável por armazenar os dados de Mogi
class Mogi():
    def __init__(self, folhosas, inmet_sp):
        self.folhosas = folhosas[folhosas['Região'] == "Mogi das Cruzes"]
        self.inmet = ProcessadorDados.process_inmet(self.folhosas, inmet_sp)
        self.folhosas_inmet = ProcessadorDados.merge_folhosas_inmet(self.folhosas, self.inmet)

In [5]:
# Classe responsável por armazenar os dados de Ibiúna
class Ibiuna():
    def __init__(self, folhosas, inmet_sorocaba):
        self.folhosas = folhosas[folhosas['Região'] == "Ibiúna"]
        self.inmet = ProcessadorDados.process_inmet(self.folhosas, inmet_sorocaba)
        self.folhosas_inmet = ProcessadorDados.merge_folhosas_inmet(self.folhosas, self.inmet)

In [ ]:
# Instância de AllDados (Salvando dados de folhosas, inmet_sp e inmet_sorocaba)
all_dados = AllDados('Folhosas.csv', 'INMET\\SP_2013_2023.csv', 'INMET\\SOROCABA_2013_2023.csv')

# Instância de Mogi (Salvando dados de mogi a partir de folhosas e inmet_sp)
mogi = Mogi(all_dados.folhosas, all_dados.inmet_sp)

# Instância de Ibiúna (Salvando dados de ibiuna a partir de folhosas e inmet_sorocaba)
ibiuna = Ibiuna(all_dados.folhosas, all_dados.inmet_sorocaba)

mogi.inmet[mogi.inmet.isna().any(axis=1)]
